In [3]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

# Path to your dataset file in Google Drive
file_path = '/content/drive/My Drive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'

# Read the dataset into a pandas DataFrame
raw = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(raw.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                   Flow ID       Source IP   Source Port  \
0  192.168.10.5-104.16.207.165-54865-443-6  104.16.207.165           443   
1    192.168.10.5-104.16.28.216-55054-80-6   104.16.28.216            80   
2    192.168.10.5-104.16.28.216-55055-80-6   104.16.28.216            80   
3  192.168.10.16-104.17.241.25-46236-443-6   104.17.241.25           443   
4  192.168.10.5-104.19.196.102-54863-443-6  104.19.196.102           443   

   Destination IP   Destination Port   Protocol      Timestamp  \
0    192.168.10.5              54865          6  7/7/2017 3:30   
1    192.168.10.5              55054          6  7/7/2017 3:30   
2    192.168.10.5              55055          6  7/7/2017 3:30   
3   192.168.10.16              46236          6  7/7/2017 3:30   
4    192.168.10.5              54863          6  7/7/2017 3:30   

    Flow Duration

In [4]:
df = raw.copy()
# Rename columns: replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_')

# Now, df has its column names with spaces replaced by underscores
print(df.columns)

Index(['Flow_ID', '_Source_IP', '_Source_Port', '_Destination_IP',
       '_Destination_Port', '_Protocol', '_Timestamp', '_Flow_Duration',
       '_Total_Fwd_Packets', '_Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', '_Total_Length_of_Bwd_Packets',
       '_Fwd_Packet_Length_Max', '_Fwd_Packet_Length_Min',
       '_Fwd_Packet_Length_Mean', '_Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', '_Bwd_Packet_Length_Min',
       '_Bwd_Packet_Length_Mean', '_Bwd_Packet_Length_Std', 'Flow_Bytes/s',
       '_Flow_Packets/s', '_Flow_IAT_Mean', '_Flow_IAT_Std', '_Flow_IAT_Max',
       '_Flow_IAT_Min', 'Fwd_IAT_Total', '_Fwd_IAT_Mean', '_Fwd_IAT_Std',
       '_Fwd_IAT_Max', '_Fwd_IAT_Min', 'Bwd_IAT_Total', '_Bwd_IAT_Mean',
       '_Bwd_IAT_Std', '_Bwd_IAT_Max', '_Bwd_IAT_Min', 'Fwd_PSH_Flags',
       '_Bwd_PSH_Flags', '_Fwd_URG_Flags', '_Bwd_URG_Flags',
       '_Fwd_Header_Length', '_Bwd_Header_Length', 'Fwd_Packets/s',
       '_Bwd_Packets/s', '_Min_Packet_Length', '_Max_Pa

In [5]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.model_selection import train_test_split

# # Assuming df is your DataFrame

# # Convert Timestamp to datetime and set as index
# df['_Timestamp'] = pd.to_datetime(df['_Timestamp'])
# df.set_index('_Timestamp', inplace=True)

# # Drop columns that are not useful for the model
# df.drop(['Flow_ID', '_Source_IP', '_Destination_IP'], axis=1, inplace=True)

# # Normalize all numeric columns
# scaler = StandardScaler()
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# # Encode categorical labels if your task is classification
# label_encoder = LabelEncoder()
# df['_Label'] = label_encoder.fit_transform(df['_Label'])
# #
# # Create sequences (example: 10 time steps per sequence)
# window_size = 10
# sequences = []
# labels = []
# for i in range(window_size, len(df)):
#     seq = df.iloc[i-window_size:i].drop('_Label', axis=1)
#     label = df.iloc[i]['_Label']
#     sequences.append(seq.values)
#     labels.append(label)

# X = np.array(sequences)
# y = np.array(labels)

# # Split X and y into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
df.columns

Index(['Flow_ID', '_Source_IP', '_Source_Port', '_Destination_IP',
       '_Destination_Port', '_Protocol', '_Timestamp', '_Flow_Duration',
       '_Total_Fwd_Packets', '_Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', '_Total_Length_of_Bwd_Packets',
       '_Fwd_Packet_Length_Max', '_Fwd_Packet_Length_Min',
       '_Fwd_Packet_Length_Mean', '_Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', '_Bwd_Packet_Length_Min',
       '_Bwd_Packet_Length_Mean', '_Bwd_Packet_Length_Std', 'Flow_Bytes/s',
       '_Flow_Packets/s', '_Flow_IAT_Mean', '_Flow_IAT_Std', '_Flow_IAT_Max',
       '_Flow_IAT_Min', 'Fwd_IAT_Total', '_Fwd_IAT_Mean', '_Fwd_IAT_Std',
       '_Fwd_IAT_Max', '_Fwd_IAT_Min', 'Bwd_IAT_Total', '_Bwd_IAT_Mean',
       '_Bwd_IAT_Std', '_Bwd_IAT_Max', '_Bwd_IAT_Min', 'Fwd_PSH_Flags',
       '_Bwd_PSH_Flags', '_Fwd_URG_Flags', '_Bwd_URG_Flags',
       '_Fwd_Header_Length', '_Bwd_Header_Length', 'Fwd_Packets/s',
       '_Bwd_Packets/s', '_Min_Packet_Length', '_Max_Pa

In [7]:
# Count the number of rows for each unique label
label_counts = df['_Label'].value_counts()
label_counts

DDoS      128027
BENIGN     97718
Name: _Label, dtype: int64

TRANSFORMER - BINARY

In [8]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_curve
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D

# Preprocess the dataset

# Drop non-numeric columns for simplicity
df.drop(['Flow_ID', '_Source_IP', '_Destination_IP', '_Timestamp'], axis=1, inplace=True)

# Encode labels
label_encoder = LabelEncoder()
df['_Label'] = label_encoder.fit_transform(df['_Label'])

# Replace inf/-inf with NaN
df = df.replace([np.inf, -np.inf], np.nan)

# Check for any remaining NaN values
if df.isnull().values.any():
    # Handle NaN values, e.g., replace them with the mean of each column
    df.fillna(df.mean(), inplace=True)

# Now you can proceed with further preprocessing like scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Assuming you want to scale the entire dataframe
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('_Label', axis=1))
y = df['_Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transformer block
def transformer_block(inputs, num_heads, ff_dim, rate=0.1):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)
    attn_output = Dropout(rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    ffn_output = Dense(ff_dim, activation="relu")(out1)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(rate)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Build the model
def build_model(input_shape, num_heads, ff_dim):
    inputs = Input(shape=(1, input_shape[0]))  # Adjust the input shape for transformer
    x = transformer_block(inputs, num_heads, ff_dim)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Adjust the training data shape
X_train_transformed = np.expand_dims(X_train, axis=1)
X_test_transformed = np.expand_dims(X_test, axis=1)

model = build_model(X_train.shape[1:], 1, 24)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
history = model.fit(X_train_transformed, y_train, batch_size=32, epochs=10, validation_split=0.2)
training_time = time.time() - start_time

# Evaluate the model
eval_results = model.evaluate(X_test_transformed, y_test)
y_pred = model.predict(X_test_transformed)
y_pred_class = (y_pred > 0.5).astype('int32')

# Calculate TPR, FPR, Precision, and F1 Score
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
precision = precision_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)

# Save or display the results
performance_metrics = {
    'True Positive Rate': tpr,
    'False Positive Rate': fpr,
    'Precision': precision,
    'F1 Score': f1,
    'Training Time (seconds)': training_time,
    'Model Evaluation Loss': eval_results[0],
    'Model Evaluation Accuracy': eval_results[1]
}

print(performance_metrics)


Epoch 1/10
4515/4515 [==============================] - 38s 7ms/step - loss: 0.0111 - accuracy: 0.9966 - val_loss: 0.0035 - val_accuracy: 0.9990
Epoch 2/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.0032 - val_accuracy: 0.9990
Epoch 3/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.0018 - val_accuracy: 0.9995
Epoch 4/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0047 - val_accuracy: 0.9988
Epoch 5/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0021 - val_accuracy: 0.9995
Epoch 6/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.0021 - val_accuracy: 0.9994
Epoch 7/10
4515/4515 [==============================] - 32s 7ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0018 - val_ac

DEEP NEURAL NETWORK - BINARY

In [9]:
from sklearn.metrics import confusion_matrix, precision_score, f1_score

def calculate_performance_metrics(tp, fp, tn, fn, training_time, eval_results):
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    precision = precision_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)

    return {
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'F1 Score': f1,
        'Training Time (seconds)': training_time,
        'Model Evaluation Loss': eval_results[0],
        'Model Evaluation Accuracy': eval_results[1]
    }

# Use this function in your model evaluation code

# Build the DNN model
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
dnn_history = dnn_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
dnn_training_time = time.time() - start_time

# Evaluate the model and calculate metrics
dnn_eval_results = dnn_model.evaluate(X_test, y_test)
dnn_y_pred = (dnn_model.predict(X_test) > 0.5).astype('int32')

# Calculate metrics
tn, fp, fn, tp = confusion_matrix(y_test, dnn_y_pred).ravel()
dnn_performance = calculate_performance_metrics(tp, fp, tn, fn, dnn_training_time, dnn_eval_results)

print("DNN Performance Metrics:", dnn_performance)

Epoch 1/10
4515/4515 [==============================] - 17s 3ms/step - loss: 0.0122 - accuracy: 0.9972 - val_loss: 0.0049 - val_accuracy: 0.9990
Epoch 2/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0047 - accuracy: 0.9989 - val_loss: 0.0034 - val_accuracy: 0.9993
Epoch 3/10
4515/4515 [==============================] - 16s 3ms/step - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 4/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0033 - accuracy: 0.9991 - val_loss: 0.0027 - val_accuracy: 0.9994
Epoch 5/10
4515/4515 [==============================] - 16s 3ms/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.0104 - val_accuracy: 0.9985
Epoch 6/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0031 - accuracy: 0.9993 - val_loss: 0.0022 - val_accuracy: 0.9995
Epoch 7/10
4515/4515 [==============================] - 15s 3ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 0.0024 - val_ac

CONVOLUTIONAL NEURAL NETWORK - BINARY

In [10]:
# Reshape input for CNN
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the CNN model
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
cnn_history = cnn_model.fit(X_train_cnn, y_train, batch_size=32, epochs=10, validation_split=0.2)
cnn_training_time = time.time() - start_time

# Evaluate the model and calculate metrics
cnn_eval_results = cnn_model.evaluate(X_test_cnn, y_test)
cnn_y_pred = (cnn_model.predict(X_test_cnn) > 0.5).astype('int32')

# Calculate metrics
tn, fp, fn, tp = confusion_matrix(y_test, cnn_y_pred).ravel()
cnn_performance = calculate_performance_metrics(tp, fp, tn, fn, cnn_training_time, cnn_eval_results)

print("CNN Performance Metrics:", cnn_performance)

Epoch 1/10
4515/4515 [==============================] - 19s 4ms/step - loss: 0.0119 - accuracy: 0.9971 - val_loss: 0.0073 - val_accuracy: 0.9984
Epoch 2/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0050 - accuracy: 0.9987 - val_loss: 0.0028 - val_accuracy: 0.9993
Epoch 3/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0025 - val_accuracy: 0.9993
Epoch 4/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.0020 - val_accuracy: 0.9993
Epoch 5/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 6/10
4515/4515 [==============================] - 16s 4ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0022 - val_accuracy: 0.9993
Epoch 7/10
4515/4515 [==============================] - 16s 3ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.0038 - val_ac

LONG SHORT TERM MEMORY - BINARY

In [11]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, f1_score
import time

# Reshape input for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(1, X_train.shape[1])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
lstm_history = lstm_model.fit(X_train_lstm, y_train, batch_size=32, epochs=10, validation_split=0.2)
lstm_training_time = time.time() - start_time

# Evaluate the model
lstm_eval_results = lstm_model.evaluate(X_test_lstm, y_test)
lstm_y_pred = (lstm_model.predict(X_test_lstm) > 0.5).astype('int32')

# Define the calculate_performance_metrics function
def calculate_performance_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return {
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'F1 Score': f1,
        'Training Time (seconds)': lstm_training_time,
        'Model Evaluation Loss': lstm_eval_results[0],
        'Model Evaluation Accuracy': lstm_eval_results[1]
    }

# Calculate performance metrics
lstm_performance = calculate_performance_metrics(y_test, lstm_y_pred)

print("LSTM Performance Metrics:", lstm_performance)


Epoch 1/10
4515/4515 [==============================] - 26s 5ms/step - loss: 0.0161 - accuracy: 0.9968 - val_loss: 0.0037 - val_accuracy: 0.9989
Epoch 2/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0025 - val_accuracy: 0.9992
Epoch 3/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.0023 - val_accuracy: 0.9992
Epoch 4/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.0030 - val_accuracy: 0.9991
Epoch 5/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.0016 - val_accuracy: 0.9994
Epoch 6/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.0016 - val_accuracy: 0.9995
Epoch 7/10
4515/4515 [==============================] - 23s 5ms/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.0014 - val_ac

In [12]:
# Define the calculate_performance_metrics function
def calculate_performance_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return {
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'F1 Score': f1,
        'Training Time (seconds)': lstm_training_time,
        'Model Evaluation Loss': lstm_eval_results[0],
        'Model Evaluation Accuracy': lstm_eval_results[1]
    }

# Calculate performance metrics
lstm_performance = calculate_performance_metrics(y_test, lstm_y_pred)

print("LSTM Performance Metrics:", lstm_performance)


LSTM Performance Metrics: {'True Positive Rate': 0.9998834679925419, 'False Positive Rate': 0.0013913939706261273, 'Precision': 0.9989521887612542, 'F1 Score': 0.9994176114303462, 'Training Time (seconds)': 229.7837643623352, 'Model Evaluation Loss': 0.0019444459741562605, 'Model Evaluation Accuracy': 0.999655527420044}


PREPARATION FOR MULTI LABEL

In [13]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

# Path to your dataset file in Google Drive
file_path = '/content/drive/My Drive/Wednesday-workingHours.pcap_ISCX.csv' #BENIGN              440031 DoS Hulk            231073 DoS GoldenEye        10293 DoS slowloris         5796 DoS Slowhttptest      5499 Heartbleed              11
# file_path = '/content/drive/My Drive/Tuesday-WorkingHours.pcap_ISCX.csv' #BENIGN         432074 FTP-Patator      7938 SSH-Patator      5897
# file_path = '/content/drive/My Drive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv' #BENIGN          288566 Infiltration        36
# file_path = '/content/drive/My Drive/Monday-WorkingHours.pcap_ISCX.csv' #BENIGN    529918
# file_path = '/content/drive/My Drive/Friday-WorkingHours-Morning.pcap_ISCX.csv' # BENIGN    189067 Bot         1966
# file_path = '/content/drive/My Drive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv' #PortScan    158930 BENIGN      127537

# Read the dataset into a pandas DataFrame
raw = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(raw.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                  Flow ID      Source IP   Source Port  \
0  192.168.10.14-209.48.71.168-49459-80-6  192.168.10.14         49459   
1  192.168.10.3-192.168.10.17-389-49453-6  192.168.10.17         49453   
2   192.168.10.3-192.168.10.17-88-46124-6  192.168.10.17         46124   
3  192.168.10.3-192.168.10.17-389-49454-6  192.168.10.17         49454   
4   192.168.10.3-192.168.10.17-88-46126-6  192.168.10.17         46126   

   Destination IP   Destination Port   Protocol      Timestamp  \
0   209.48.71.168                 80          6  5/7/2017 8:42   
1    192.168.10.3                389          6  5/7/2017 8:42   
2    192.168.10.3                 88          6  5/7/2017 8:42   
3    192.168.10.3                389          6  5/7/2017 8:42   
4    192.168.10.3                 88          6  5/7/2017 8:42   

    Flow Duration   Total Fwd

In [14]:
df = raw.copy()
# Rename columns: replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_')

df = df[df['_Label'] != 'Heartbleed']

# Now, df has its column names with spaces replaced by underscores
print(df.columns)

Index(['Flow_ID', '_Source_IP', '_Source_Port', '_Destination_IP',
       '_Destination_Port', '_Protocol', '_Timestamp', '_Flow_Duration',
       '_Total_Fwd_Packets', '_Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', '_Total_Length_of_Bwd_Packets',
       '_Fwd_Packet_Length_Max', '_Fwd_Packet_Length_Min',
       '_Fwd_Packet_Length_Mean', '_Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', '_Bwd_Packet_Length_Min',
       '_Bwd_Packet_Length_Mean', '_Bwd_Packet_Length_Std', 'Flow_Bytes/s',
       '_Flow_Packets/s', '_Flow_IAT_Mean', '_Flow_IAT_Std', '_Flow_IAT_Max',
       '_Flow_IAT_Min', 'Fwd_IAT_Total', '_Fwd_IAT_Mean', '_Fwd_IAT_Std',
       '_Fwd_IAT_Max', '_Fwd_IAT_Min', 'Bwd_IAT_Total', '_Bwd_IAT_Mean',
       '_Bwd_IAT_Std', '_Bwd_IAT_Max', '_Bwd_IAT_Min', 'Fwd_PSH_Flags',
       '_Bwd_PSH_Flags', '_Fwd_URG_Flags', '_Bwd_URG_Flags',
       '_Fwd_Header_Length', '_Bwd_Header_Length', 'Fwd_Packets/s',
       '_Bwd_Packets/s', '_Min_Packet_Length', '_Max_Pa

In [15]:
# Count the number of rows for each unique label
label_counts = df['_Label'].value_counts()
label_counts

BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Name: _Label, dtype: int64

TRANSFORMER - MULTI LABEL

In [16]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_curve
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming df is your DataFrame

# One-hot encode the labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['_Label'])
y = to_categorical(integer_encoded)

# You may want to drop non-numeric columns for simplicity, but consider if any could be useful for your task
df.drop(['Flow_ID', '_Source_IP', '_Destination_IP', '_Timestamp'], axis=1, inplace=True)

# Replace inf/-inf with NaN and handle NaN values
df = df.replace([np.inf, -np.inf], np.nan)
df.fillna(df.mean(), inplace=True)

# Prepare the feature matrix X
X = df.drop('_Label', axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Transformer block
def transformer_block(inputs, num_heads, ff_dim, rate=0.1):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)
    attn_output = Dropout(rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    ffn_output = Dense(ff_dim, activation="relu")(out1)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(rate)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

def build_model(input_shape, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(1, input_shape))  # Adjusted input shape
    x = transformer_block(inputs, num_heads, ff_dim)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)  # Output layer for multi-label

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model configuration
num_classes = y_train_smote.shape[1]
model = build_model(X_train_scaled.shape[1], num_heads=3, ff_dim=24, num_classes=num_classes)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Adjust the training data shape for the Transformer model
X_train_transformed = np.expand_dims(X_train_scaled, axis=1)
X_test_transformed = np.expand_dims(X_test_scaled, axis=1)

# Initialize EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transformer_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model with more epochs and callbacks for early stopping and model checkpoint
start_time = time.time()
history = model.fit(
    X_train_transformed, y_train_smote,
    batch_size=32,
    epochs=50,  # Increase the number of epochs
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]  # Add callbacks
)
training_time = time.time() - start_time

# Evaluate the model
eval_results = model.evaluate(X_test_transformed, y_test)

print(f"Training Time: {training_time} seconds")


<ipython-input-16-7366335ead40>:26: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


Epoch 1/50
43995/43995 [==============================] - ETA: 0s - loss: 0.0133 - accuracy: 0.9904
Epoch 1: val_loss improved from inf to 0.07922, saving model to best_transformer_model.h5
43995/43995 [==============================] - 236s 5ms/step - loss: 0.0133 - accuracy: 0.9904 - val_loss: 0.0792 - val_accuracy: 0.9225
Epoch 2/50
   22/43995 [..............................] - ETA: 3:41 - loss: 0.0086 - accuracy: 0.9943

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43992/43995 [============================>.] - ETA: 0s - loss: 0.0086 - accuracy: 0.9939
Epoch 2: val_loss did not improve from 0.07922
43995/43995 [==============================] - 233s 5ms/step - loss: 0.0086 - accuracy: 0.9939 - val_loss: 0.1860 - val_accuracy: 0.8630
Epoch 3/50
43989/43995 [============================>.] - ETA: 0s - loss: 0.0069 - accuracy: 0.9952
Epoch 3: val_loss did not improve from 0.07922
43995/43995 [==============================] - 234s 5ms/step - loss: 0.0069 - accuracy: 0.9952 - val_loss: 0.0871 - val_accuracy: 0.9362
Epoch 4/50
43985/43995 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.9959
Epoch 4: val_loss improved from 0.07922 to 0.05109, saving model to best_transformer_model.h5
43995/43995 [==============================] - 233s 5ms/step - loss: 0.0061 - accuracy: 0.9959 - val_loss: 0.0511 - val_accuracy: 0.9653
Epoch 5/50
43992/43995 [============================>.] - ETA: 0s - loss: 0.0055 - accuracy: 0.9964
Epoch 5: val

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Function to calculate TPR and FPR for each label
def calculate_tpr_fpr_per_label(y_true, y_pred):
    tpr_list = []
    fpr_list = []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return tpr_list, fpr_list

# Function to calculate performance metrics for each label
def calculate_performance_metrics_per_label(y_true, y_pred):
    tpr_list, fpr_list = calculate_tpr_fpr_per_label(y_true, y_pred)
    precision_list = []
    recall_list = []
    f1_list = []

    for i in range(y_true.shape[1]):
        precision = precision_score(y_true[:, i], y_pred[:, i])
        recall = recall_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return {
        'True Positive Rate': tpr_list,
        'False Positive Rate': fpr_list,
        'Precision': precision_list,
        'Recall': recall_list,
        'F1 Score': f1_list
    }

# Predictions for multi-label classification
y_pred = model.predict(X_test_transformed)
y_pred_class = (y_pred > 0.5).astype('int32')

# Calculate performance metrics
metrics_per_label = calculate_performance_metrics_per_label(y_test, y_pred_class)
metrics_per_label['Training Time (seconds)'] = training_time
metrics_per_label['Model Evaluation Loss'] = eval_results[0]
metrics_per_label['Model Evaluation Accuracy'] = eval_results[1]

# Print or save the results
print("Transformer Performance Metrics Per Label:", metrics_per_label)

4330/4330 [==============================] - 10s 2ms/step
Transformer Performance Metrics Per Label: {'True Positive Rate': [0.9857501334150855, 0.998533724340176, 0.9988316240425809, 0.9944954128440368, 0.9452423698384201], 'False Positive Rate': [0.000495363398589205, 0.000798575751137421, 0.011167556677245697, 0.0010913138691441917, 8.732035655812261e-05], 'Precision': [0.9997121175481627, 0.9493494423791822, 0.9781544655154147, 0.8784440842787682, 0.9887323943661972], 'Recall': [0.9857501334150855, 0.998533724340176, 0.9988316240425809, 0.9944954128440368, 0.9452423698384201], 'F1 Score': [0.9926820343944384, 0.9733206288708909, 0.9883849143052894, 0.9328743545611016, 0.9664983937586048], 'Training Time (seconds)': 2101.0584704875946, 'Model Evaluation Loss': 0.011089515872299671, 'Model Evaluation Accuracy': 0.9903131723403931}


In [18]:
# Fit the LabelEncoder to the labels
label_encoder = LabelEncoder()
label_encoder.fit(df['_Label'])

# Get the list of original class names
label_names = label_encoder.classes_


from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def calculate_metrics_per_label(y_true, y_pred, label_names):
    metrics = {}
    for i, label in enumerate(label_names):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()

        # Calculating TPR and FPR
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0

        # Calculating Precision and F1 Score
        precision = precision_score(y_true[:, i], y_pred[:, i], zero_division=0)
        f1 = f1_score(y_true[:, i], y_pred[:, i], zero_division=0)

        metrics[label] = {'TPR': tpr, 'FPR': fpr, 'Precision': precision, 'F1 Score': f1}
    return metrics

def create_confusion_matrix_grid(y_true, y_pred, label_names):
    # Convert label_names to a list if it's not already
    if isinstance(label_names, np.ndarray):
        label_names = label_names.tolist()

    # Initialize an empty DataFrame to store the confusion matrix grid
    confusion_matrix_grid = pd.DataFrame(index=label_names, columns=label_names)

    for actual_label in label_names:
        actual_index = label_names.index(actual_label)
        for predicted_label in label_names:
            predicted_index = label_names.index(predicted_label)

            # Calculate the count for actual vs predicted label
            count = sum((y_true[:, actual_index] == 1) & (y_pred[:, predicted_index] == 1))
            confusion_matrix_grid.at[actual_label, predicted_label] = count

    return confusion_matrix_grid


# Example usage
metrics_per_label = calculate_metrics_per_label(y_test, y_pred_class, label_names)



In [19]:
# Print the corresponding label names
print("Label Encoding Order:", label_names)

Label Encoding Order: ['BENIGN' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest' 'DoS slowloris']


In [20]:
metrics_per_label

{'BENIGN': {'TPR': 0.9857501334150855,
  'FPR': 0.000495363398589205,
  'Precision': 0.9997121175481627,
  'F1 Score': 0.9926820343944384},
 'DoS GoldenEye': {'TPR': 0.998533724340176,
  'FPR': 0.000798575751137421,
  'Precision': 0.9493494423791822,
  'F1 Score': 0.9733206288708909},
 'DoS Hulk': {'TPR': 0.9988316240425809,
  'FPR': 0.011167556677245697,
  'Precision': 0.9781544655154147,
  'F1 Score': 0.9883849143052894},
 'DoS Slowhttptest': {'TPR': 0.9944954128440368,
  'FPR': 0.0010913138691441917,
  'Precision': 0.8784440842787682,
  'F1 Score': 0.9328743545611016},
 'DoS slowloris': {'TPR': 0.9452423698384201,
  'FPR': 8.732035655812261e-05,
  'Precision': 0.9887323943661972,
  'F1 Score': 0.9664983937586048}}

In [21]:
confusion_matrix_grid = create_confusion_matrix_grid(y_test, y_pred_class, label_names)

print(confusion_matrix_grid)

                 BENIGN DoS GoldenEye DoS Hulk DoS Slowhttptest DoS slowloris
BENIGN            86816            74     1029              106            11
DoS GoldenEye         0          2043        1                2             0
DoS Hulk             20            31    46164                0             0
DoS Slowhttptest      3             3        0             1084             1
DoS slowloris         2             1        1               42          1053


In [22]:

# Function to calculate TPR and FPR
def calculate_tpr_fpr(y_true, y_pred):
    # Calculate confusion matrix for each label
    tpr_list = []
    fpr_list = []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)

    # Return average TPR and FPR
    return np.mean(tpr_list), np.mean(fpr_list)

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
# Function to calculate performance metrics
def calculate_performance_metrics(y_true, y_pred):
    # Calculate TPR and FPR
    tpr, fpr = calculate_tpr_fpr(y_true, y_pred)

    # Calculate precision, recall, and F1 score for each label and find their average
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    return {
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Predictions for multi-label classification
y_pred = model.predict(X_test_transformed)
y_pred_class = (y_pred > 0.5).astype('int32')

# Calculate performance metrics
metrics = calculate_performance_metrics(y_test, y_pred_class)
metrics['Training Time (seconds)'] = training_time
metrics['Model Evaluation Loss'] = eval_results[0]
metrics['Model Evaluation Accuracy'] = eval_results[1]

# Print or save the results
print("Transformer Performance Metrics:", metrics)

4330/4330 [==============================] - 10s 2ms/step
Transformer Performance Metrics: {'True Positive Rate': 0.9988906528960598, 'False Positive Rate': 0.00103280260105349277, 'Precision': 0.9985578731577693, 'Recall': 0.9964461241960747, 'F1 Score': 0.9987019637867926, 'Training Time (seconds)': 2101.0584704875946, 'Model Evaluation Loss': 0.0033789515872299671, 'Model Evaluation Accuracy': 0.996681723403931}


In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Function to calculate TPR and FPR for each label
def calculate_tpr_fpr_per_label(y_true, y_pred):
    tpr_list = []
    fpr_list = []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return tpr_list, fpr_list

# Function to calculate performance metrics for each label
def calculate_performance_metrics_per_label(y_true, y_pred):
    tpr_list, fpr_list = calculate_tpr_fpr_per_label(y_true, y_pred)
    precision_list = []
    recall_list = []
    f1_list = []

    for i in range(y_true.shape[1]):
        precision = precision_score(y_true[:, i], y_pred[:, i])
        recall = recall_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return {
        'True Positive Rate': tpr_list,
        'False Positive Rate': fpr_list,
        'Precision': precision_list,
        'Recall': recall_list,
        'F1 Score': f1_list
    }

# Predictions for multi-label classification
y_pred = model.predict(X_test_transformed)
y_pred_class = (y_pred > 0.5).astype('int32')

# Calculate performance metrics
metrics_per_label = calculate_performance_metrics_per_label(y_test, y_pred_class)
metrics_per_label['Training Time (seconds)'] = training_time
metrics_per_label['Model Evaluation Loss'] = eval_results[0]
metrics_per_label['Model Evaluation Accuracy'] = eval_results[1]

# Print or save the results
print("Transformer Performance Metrics Per Label:", metrics_per_label)

4330/4330 [==============================] - 10s 2ms/step
Transformer Performance Metrics Per Label: {'True Positive Rate': [0.9857501334150855, 0.998533724340176, 0.9988316240425809, 0.9944954128440368, 0.9452423698384201], 'False Positive Rate': [0.000495363398589205, 0.000798575751137421, 0.011167556677245697, 0.0010913138691441917, 8.732035655812261e-05], 'Precision': [0.9997121175481627, 0.9493494423791822, 0.9781544655154147, 0.8784440842787682, 0.9887323943661972], 'Recall': [0.9857501334150855, 0.998533724340176, 0.9988316240425809, 0.9944954128440368, 0.9452423698384201], 'F1 Score': [0.9926820343944384, 0.9733206288708909, 0.9883849143052894, 0.9328743545611016, 0.9664983937586048], 'Training Time (seconds)': 2101.0584704875946, 'Model Evaluation Loss': 0.011089515872299671, 'Model Evaluation Accuracy': 0.9903131723403931}


DEEP NEURAL NETWORK - MULTI LABEL

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss
import tensorflow as tf
import time

# Assume df is your DataFrame
# Convert categorical labels to a format suitable for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df[['_Label']].values)

# Identify feature columns
feature_columns = df.columns.drop(['_Label'])

# Drop non-relevant columns and preprocess features
scaler = StandardScaler()
X = scaler.fit_transform(df[feature_columns])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the DNN Model
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')  # Output layer for multi-label
])

dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
dnn_history = dnn_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
dnn_training_time = time.time() - start_time

# Evaluate the model
dnn_eval_results = dnn_model.evaluate(X_test, y_test)
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Function to calculate TPR and FPR for multi-label classification
def calculate_tpr_fpr(y_true, y_pred):
    tpr_list = []
    fpr_list = []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return np.mean(tpr_list), np.mean(fpr_list)

# Function to calculate performance metrics for multi-label classification
def calculate_performance_metrics(y_true, y_pred, training_time, eval_results):
    tpr, fpr = calculate_tpr_fpr(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    return {
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Training Time (seconds)': training_time,
        'Model Evaluation Loss': eval_results[0],
        'Model Evaluation Accuracy': eval_results[1]
    }

# Predict and calculate multi-label metrics for DNN
dnn_y_pred = (dnn_model.predict(X_test) > 0.5).astype('int32')
dnn_performance = calculate_performance_metrics(y_test, dnn_y_pred, dnn_training_time, dnn_eval_results)

print("DNN Performance Metrics:", dnn_performance)

Epoch 1/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0190 - accuracy: 0.9842 - val_loss: 0.0119 - val_accuracy: 0.9899
Epoch 2/10
13854/13854 [==============================] - 45s 3ms/step - loss: 0.0123 - accuracy: 0.9896 - val_loss: 0.0098 - val_accuracy: 0.9911
Epoch 3/10
13854/13854 [==============================] - 44s 3ms/step - loss: 0.0114 - accuracy: 0.9901 - val_loss: 0.0108 - val_accuracy: 0.9913
Epoch 4/10
13854/13854 [==============================] - 45s 3ms/step - loss: 0.0109 - accuracy: 0.9905 - val_loss: 0.0090 - val_accuracy: 0.9915
Epoch 5/10
13854/13854 [==============================] - 45s 3ms/step - loss: 0.0103 - accuracy: 0.9907 - val_loss: 0.0087 - val_accuracy: 0.9917
Epoch 6/10
13854/13854 [==============================] - 45s 3ms/step - loss: 0.0102 - accuracy: 0.9910 - val_loss: 0.0083 - val_accuracy: 0.9918
Epoch 7/10
13854/13854 [==============================] - 45s 3ms/step - loss: 0.0100 - accuracy: 0.9912 - val_loss: 0

CONVOLUTIONAL NEURAL NETWORK - MULTI LABEL

In [25]:
# Reshape input for CNN
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the CNN Model
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')  # Output layer for multi-label
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN Model
start_time = time.time()
cnn_history = cnn_model.fit(X_train_cnn, y_train, batch_size=32, epochs=10, validation_split=0.2)
cnn_training_time = time.time() - start_time

# Evaluate the CNN Model
cnn_eval_results = cnn_model.evaluate(X_test_cnn, y_test)

# Predict and calculate multi-label metrics for CNN
cnn_y_pred = (cnn_model.predict(X_test_cnn) > 0.5).astype('int32')

Epoch 1/10
13854/13854 [==============================] - 47s 3ms/step - loss: 0.0178 - accuracy: 0.9855 - val_loss: 0.0123 - val_accuracy: 0.9909
Epoch 2/10
13854/13854 [==============================] - 47s 3ms/step - loss: 0.0111 - accuracy: 0.9906 - val_loss: 0.0096 - val_accuracy: 0.9913
Epoch 3/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0100 - accuracy: 0.9914 - val_loss: 0.0086 - val_accuracy: 0.9916
Epoch 4/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0089 - accuracy: 0.9924 - val_loss: 0.0081 - val_accuracy: 0.9945
Epoch 5/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0078 - accuracy: 0.9938 - val_loss: 0.0079 - val_accuracy: 0.9945
Epoch 6/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0068 - accuracy: 0.9950 - val_loss: 0.0075 - val_accuracy: 0.9946
Epoch 7/10
13854/13854 [==============================] - 46s 3ms/step - loss: 0.0063 - accuracy: 0.9957 - val_loss: 0

In [26]:
import numpy as np
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score, confusion_matrix

def calculate_tpr_fpr(y_true, y_pred):
    tpr_list = []
    fpr_list = []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return np.mean(tpr_list), np.mean(fpr_list)

def calculate_performance_metrics(y_true, y_pred, training_time, eval_results):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate Hamming loss
    ham_loss = hamming_loss(y_true, y_pred)

    # Calculate micro-averaged Precision, Recall, and F1 Score for multi-label classification
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    # Calculate average TPR and FPR for multi-label classification
    tpr, fpr = calculate_tpr_fpr(y_true, y_pred)

    # Collecting all metrics
    performance_metrics = {
        'Accuracy': accuracy,
        'Hamming Loss': ham_loss,
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Training Time (seconds)': training_time,
        'Model Evaluation Loss': eval_results[0],
        'Model Evaluation Accuracy': eval_results[1]
    }

    return performance_metrics

# Assuming cnn_y_pred, cnn_training_time, cnn_eval_results are already defined
cnn_performance = calculate_performance_metrics(y_test, cnn_y_pred, cnn_training_time, cnn_eval_results)
print("CNN Performance Metrics:", cnn_performance)


CNN Performance Metrics: {'Accuracy': 0.9964847443680119, 'Hamming Loss': 0.0013469131435913354, 'True Positive Rate': 0.9913748655728847, 'False Positive Rate': 0.0010880991146476483, 'Precision': 0.996578951167054, 'Recall': 0.9966868535213911, 'F1 Score': 0.9966328994236592, 'Training Time (seconds)': 462.691410779953, 'Model Evaluation Loss': 0.0054249232634902, 'Model Evaluation Accuracy': 0.9966291189193726}


LONG SHORT TERM MEMORY - MULTI LABEL

In [27]:
# Reshape input for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM Model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(1, X_train.shape[1])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')  # Output layer for multi-label
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the LSTM Model
start_time = time.time()
lstm_history = lstm_model.fit(X_train_lstm, y_train, batch_size=32, epochs=10, validation_split=0.2)
lstm_training_time = time.time() - start_time

# Evaluate the LSTM Model
lstm_eval_results = lstm_model.evaluate(X_test_lstm, y_test)

# Predict and calculate multi-label metrics for LSTM
lstm_y_pred = (lstm_model.predict(X_test_lstm) > 0.5).astype('int32')
lstm_performance = calculate_performance_metrics(y_test, lstm_y_pred, lstm_training_time, lstm_eval_results)

print("LSTM Performance Metrics:", lstm_performance)


Epoch 1/10
13854/13854 [==============================] - 73s 5ms/step - loss: 0.0183 - accuracy: 0.9857 - val_loss: 0.0099 - val_accuracy: 0.9909
Epoch 2/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0100 - accuracy: 0.9909 - val_loss: 0.0090 - val_accuracy: 0.9917
Epoch 3/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0093 - accuracy: 0.9913 - val_loss: 0.0112 - val_accuracy: 0.9797
Epoch 4/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0087 - accuracy: 0.9918 - val_loss: 0.0085 - val_accuracy: 0.9919
Epoch 5/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0082 - accuracy: 0.9922 - val_loss: 0.0077 - val_accuracy: 0.9919
Epoch 6/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0077 - accuracy: 0.9928 - val_loss: 0.0087 - val_accuracy: 0.9922
Epoch 7/10
13854/13854 [==============================] - 69s 5ms/step - loss: 0.0073 - accuracy: 0.9933 - val_loss: 0

In [28]:
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score, confusion_matrix

def calculate_tpr_fpr(y_true, y_pred):
    # Calculate TPR and FPR for each label and average them
    tpr_list, fpr_list = [], []
    for i in range(y_true.shape[1]):
        tn, fp, fn, tp = confusion_matrix(y_true[:, i], y_pred[:, i]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return np.mean(tpr_list), np.mean(fpr_list)

def calculate_performance_metrics(y_true, y_pred, training_time, eval_results):
    # Standard metrics
    accuracy = accuracy_score(y_true, y_pred)
    ham_loss = hamming_loss(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    # Additional metrics
    tpr, fpr = calculate_tpr_fpr(y_true, y_pred)

    # Metrics collection
    performance_metrics = {
        'Accuracy': accuracy,
        'Hamming Loss': ham_loss,
        'True Positive Rate': tpr,
        'False Positive Rate': fpr,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Training Time (seconds)': training_time,
        'Model Evaluation Loss': eval_results[0],
        'Model Evaluation Accuracy': eval_results[1]
    }

    return performance_metrics

# Predict and calculate multi-label metrics for LSTM
lstm_y_pred = (lstm_model.predict(X_test_lstm) > 0.5).astype('int32')
lstm_performance = calculate_performance_metrics(y_test, lstm_y_pred, lstm_training_time, lstm_eval_results)

print("LSTM Performance Metrics:", lstm_performance)


4330/4330 [==============================] - 8s 2ms/step
LSTM Performance Metrics: {'Accuracy': 0.9951926894231949, 'Hamming Loss': 0.001850742390229466, 'True Positive Rate': 0.9870233450182557, 'False Positive Rate': 0.001613543299618896, 'Precision': 0.9954089035508811, 'Recall': 0.9953370531041801, 'F1 Score': 0.9953729770309094, 'Training Time (seconds)': 693.9536406993866, 'Model Evaluation Loss': 0.005764384288340807, 'Model Evaluation Accuracy': 0.9954525232315063}
